In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

c:\Users\ultra\AppData\Local\Programs\Python\Python37\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Reusing dataset glue (C:\Users\ultra\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 745.92it/s]
Loading cached processed dataset at C:\Users\ultra\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-75ac652e17c86003.arrow
Loading cached processed dataset at C:\Users\ultra\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-4131c84b34bfa665.arrow
100%|██████████| 2/2 [00:00<00:00, 16.35ba/s]


In [2]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])


In [3]:
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [4]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
                tokenized_datasets["train"], shuffle=True, batch_size = 32, collate_fn=data_collator
)
eval_loader = DataLoader(
    tokenized_datasets["validation"], batch_size=32, collate_fn=data_collator
    )

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [6]:
"""Testing that training will go well. Passing a batch to the model"""

for batch in train_loader:
    break
{k: v.shape for k, v in batch.items()}

outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.5588, grad_fn=<NllLossBackward0>) torch.Size([32, 2])


In [7]:
#Optimizer and Scheduler
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr = 5e-4)
epochs = 3
num_training_steps = epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps = 0,
    num_training_steps=num_training_steps,
)

num_training_steps

c:\Users\ultra\AppData\Local\Programs\Python\Python37\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


345

In [8]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [15]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
model.train()

for e in range(epochs):
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        output = model(**batch)
        loss = output.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

100%|██████████| 345/345 [02:38<00:00,  2.27it/s]

In [16]:
#Evaluation

from datasets import load_metric

metric = load_metric("glue", "mrpc")
model.eval()
for batch in eval_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.6838235294117647, 'f1': 0.8122270742358079}